In [1]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch 

import mysql.connector

In [2]:
from elasticsearch.helpers import bulk

# 读取数据

In [9]:
news_df = pd.read_csv('../data/news_data/event.csv')

In [11]:
news_df = news_df.fillna('')

In [12]:
news_df

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,MediaGeo,Title,Content
0,845139598,20180514,201805,2018,2018.3671,,,,,,...,UKH9,40110,51.5,-0.116667,-2601889,20190514120000,https://www.tradewindsnews.com/legal/1774792/u...,Norway,UK Supreme Court's shipping shortfall angers i...,UK Supreme Court's shipping shortfall angers i...
1,845139599,20180514,201805,2018,2018.3671,BUS,COMPANY,,,,...,PK04,23661,31.4167,73.0833,-2759985,20190514120000,https://www.pakwheels.com/blog/pakistanis-migh...,United States of America,Pakistanis might not see Renault vehicles in t...,Update:\nOne of the most awaited brands in Pak...
2,845139600,20180514,201805,2018,2018.3671,BUS,COMPANY,,,,...,PK04,23661,31.4167,73.0833,-2759985,20190514120000,https://www.pakwheels.com/blog/pakistanis-migh...,United States of America,Pakistanis might not see Renault vehicles in t...,Update:\nOne of the most awaited brands in Pak...
3,845139601,20180514,201805,2018,2018.3671,BUS,COMPANY,,,,...,FR,,46,2,FR,20190514120000,https://www.pakwheels.com/blog/pakistanis-migh...,United States of America,Pakistanis might not see Renault vehicles in t...,Update:\nOne of the most awaited brands in Pak...
4,845139602,20180514,201805,2018,2018.3671,COP,PRISON,,,,...,UKH9,40110,51.5,-0.116667,-2601889,20190514120000,http://islingtontribune.com/article/holloway-p...,Ireland,Holloway prison development loan from the Mayo...,Sadiq Khan at the opening of a new LGBT homele...
5,845139603,20180514,201805,2018,2018.3671,GBR,UNITED KINGDOM,GBR,,,...,UKU8,40173,55.95,-3.2,-2595386,20190514120000,https://www.dailyecho.co.uk/news/national/1763...,United Kingdom,None,None
6,845139604,20180514,201805,2018,2018.3671,GBR,UNITED KINGDOM,GBR,,,...,UKU8,40173,55.95,-3.2,-2595386,20190514120000,https://www.dailyecho.co.uk/news/national/1763...,United Kingdom,None,None
7,845139605,20180514,201805,2018,2018.3671,IGOEUREEC,THE EU,EUR,EEC,,...,CY,,35,33,CY,20190514120000,https://greece.greekreporter.com/2019/05/14/eu...,United States of America,EU Issues New Warning Against Turkey: Respect ...,"\n\nFederica Mogherini, the High Representativ..."
8,845139606,20180514,201805,2018,2018.3671,IGOEUREEC,THE EU,EUR,EEC,,...,GR,,39,22,GR,20190514120000,https://greece.greekreporter.com/2019/05/14/eu...,United States of America,EU Issues New Warning Against Turkey: Respect ...,"\n\nFederica Mogherini, the High Representativ..."
9,845139607,20180514,201805,2018,2018.3671,IGOEUREEC,THE EU,EUR,EEC,,...,GR,,39,22,GR,20190514120000,https://greece.greekreporter.com/2019/05/14/eu...,United States of America,EU Issues New Warning Against Turkey: Respect ...,"\n\nFederica Mogherini, the High Representativ..."


# 日期处理

In [3]:
def date_convert(dot_date_str):
    '''xxxx.xx.xx --> xxxx-xx-xx
    then pack into json for es
    '''
    y,m,d = dot_date_str.split('.')
    new_date = '{year}-{month}-{date}'.format(year=y,month=m,date=d)
    return new_date
#     return {new_date:{'type':'date','formate':'yyyy-mm-dd'}}

In [4]:
def date_autofill(dot_date_str):
    '''xxxx --> xxxx-01-01
    then pack into json for es
    '''
    new_date = '{year}-{month}-{date}'.format(year=dot_date_str,month='01',date='01')
    return new_date

# news表头

In [7]:
head = '`GLOBALEVENTID`, `SQLDATE`, `MonthYear`, `Year`, `FractionDate`, `Actor1Code`, `Actor1Name`, `Actor1CountryCode`, `Actor1KnownGroupCode`, `Actor1EthnicCode`, `Actor1Religion1Code`, `Actor1Religion2Code`, `Actor1Type1Code`, `Actor1Type2Code`, `Actor1Type3Code`, `Actor2Code`, `Actor2Name`, `Actor2CountryCode`, `Actor2KnownGroupCode`, `Actor2EthnicCode`, `Actor2Religion1Code`, `Actor2Religion2Code`, `Actor2Type1Code`, `Actor2Type2Code`, `Actor2Type3Code`, `IsRootEvent`, `EventCode`, `EventBaseCode`, `EventRootCode`, `QuadClass`, `GoldsteinScale`, `NumMentions`, `NumSources`, `NumArticles`, `AvgTone`, `Actor1Geo_Type`, `Actor1Geo_FullName`, `Actor1Geo_CountryCode`, `Actor1Geo_ADM1Code`, `Actor1Geo_ADM2Code`, `Actor1Geo_Lat`, `Actor1Geo_Long`, `Actor1Geo_FeatureID`, `Actor2Geo_Type`, `Actor2Geo_FullName`, `Actor2Geo_CountryCode`, `Actor2Geo_ADM1Code`, `Actor2Geo_ADM2Code`, `Actor2Geo_Lat`, `Actor2Geo_Long`, `Actor2Geo_FeatureID`, `ActionGeo_Type`, `ActionGeo_FullName`, `ActionGeo_CountryCode`, `ActionGeo_ADM1Code`, `ActionGeo_ADM2Code`, `ActionGeo_Lat`, `ActionGeo_Long`, `ActionGeo_FeatureID`, `DATEADDED`, `SOURCEURL`, `MediaGeo`, `Title`, `Content`'
head = head.replace('`','')
head = [i.strip() for i in head.split(',')]

In [14]:
news_df.iloc[0].to_json()

'{"GLOBALEVENTID":845139598,"SQLDATE":20180514,"MonthYear":201805,"Year":2018,"FractionDate":2018.3671,"Actor1Code":"","Actor1Name":"","Actor1CountryCode":"","Actor1KnownGroupCode":"","Actor1EthnicCode":"","Actor1Religion1Code":"","Actor1Religion2Code":"","Actor1Type1Code":"","Actor1Type2Code":"","Actor1Type3Code":"","Actor2Code":"GBR","Actor2Name":"UNITED KINGDOM","Actor2CountryCode":"GBR","Actor2KnownGroupCode":"","Actor2EthnicCode":"","Actor2Religion1Code":"","Actor2Religion2Code":"","Actor2Type1Code":"","Actor2Type2Code":"","Actor2Type3Code":"","IsRootEvent":0,"EventCode":353,"EventBaseCode":35,"EventRootCode":3,"QuadClass":1,"GoldsteinScale":7.0,"NumMentions":2,"NumSources":1,"NumArticles":2,"AvgTone":1.8930957684,"Actor1Geo_Type":0,"Actor1Geo_FullName":"","Actor1Geo_CountryCode":"","Actor1Geo_ADM1Code":"","Actor1Geo_ADM2Code":"","Actor1Geo_Lat":"","Actor1Geo_Long":"","Actor1Geo_FeatureID":"","Actor2Geo_Type":4,"Actor2Geo_FullName":"London, London, City of, United Kingdom","Actor2

# 插入

In [17]:
es = Elasticsearch(hosts='10.8.128.205',port=49200,timeout=30, max_retries=10, retry_on_timeout=True)

In [18]:
index_name = 'news-gdelt-index'

In [20]:
data_batch = []
for num,i in enumerate(news_df.itertuples()):
    if num % 5000==0 and not num ==0:
        print('[{0}] Epoch:{1}'.format(time.ctime(),int(num/5000)))
        bulk(es,data_batch,index=index_name,raise_on_error=True)
        data_batch = []
    else:
        data_batch.append(news_df.iloc[num].to_json())

# 最后一桶
bulk(es,data_batch,index=index_name,raise_on_error=True)

[Tue Jul 23 00:29:11 2019] Epoch:1
[Tue Jul 23 00:29:20 2019] Epoch:2
[Tue Jul 23 00:29:27 2019] Epoch:3
[Tue Jul 23 00:29:34 2019] Epoch:4


(1280, [])

# 设置ES状态为可写（触发硬盘存储超限后集群会自动配置所有索引为只读）

In [42]:
import json

import requests

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/json',
    'Origin': 'chrome-extension://ffmkiejjmecolpfloofpjologoblkegm',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

unlock_dict = {
"index.blocks.read_only_allow_delete": "false"
}

ppp = requests.put('http://10.8.128.205:49200/paper-year-index/_settings',data=json.dumps(unlock_dict),headers=headers)

# 论文检索对象